In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('data.csv', encoding='latin1', header=None)
y = pd.read_csv('labels.csv', encoding='latin1', header=None)

In [ ]:
X = np.squeeze(data.values)
y = np.squeeze(y.values)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, make_scorer
from sklearn import tree, svm, neighbors, model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np

Nao rodei experimentos para sklearn.naive_bayes.ComplementNB - está falhando o import

In [ ]:
'''
#from sklearn.naive_bayes import ComplementNB
cnb_clf = ComplementNB()
parameters = {
    "complementnb__alpha": [0, 1, 3, 5],
    "complementnb__norm": [True, False],
    
}
cnb_clf.random_state = seed
models.append(("naive_bayes.ComplementNB", cnb_clf, parameters))
'''

In [ ]:
seed = 42
np.random.seed(seed)

In [ ]:
models = []


random_forest_clf = RandomForestClassifier()
random_forest_clf.random_state = seed
parameters = {
    "randomforestclassifier__n_estimators":[10, 20, 30],
    "randomforestclassifier__max_depth": [None, 5, 10, 15, 20]
}
models.append(("RandomForestClassifier", random_forest_clf, parameters))



mnb_clf = MultinomialNB()
parameters = {
    "multinomialnb__alpha": [0, 1, 3, 5, 10]
}
mnb_clf.random_state = seed
models.append(("naive_bayes.MultinomialNB", mnb_clf, parameters))



decision_tree_clf = tree.DecisionTreeClassifier()
decision_tree_clf.random_state = seed
parameters = {
    "decisiontreeclassifier__max_depth": [None, 2, 5, 8, 10, 15]
}
models.append(("decision tree", decision_tree_clf, parameters))


logistic_regression_clf = LogisticRegression()
logistic_regression_clf.random_state = seed
parameters = {
    "logisticregression__penalty": ['l1', 'l2'],
    "logisticregression__C": [0.01, 0.1, 1, 5, 10, 15, 20]
}
models.append(("logistic regression", logistic_regression_clf, parameters))


svm_clf = svm.SVC()
svm_clf.random_state = seed
parameters = {'svc__kernel':('linear', 'rbf'), 'svc__C':[0, 1, 5, 10, 15, 20]}
models.append(("svm", svm_clf, parameters))



mlp_clf = MLPClassifier()
mlp_clf.random_state = seed

parameters = {'mlpclassifier__solver': ['lbfgs'], 
              'mlpclassifier__activation':['relu'], 
              'mlpclassifier__max_iter': [2000], 
              'mlpclassifier__alpha': 10.0 ** -np.arange(1, 10), 
              'mlpclassifier__tol':[1e-3], 
              'mlpclassifier__hidden_layer_sizes':np.arange(10, 15)}

models.append(("mlp", mlp_clf, parameters))





knn_clf = neighbors.KNeighborsClassifier()
knn_clf.random_state = seed
parameters = {
    #"kneighborsclassifier__n_neighbors": [1, 3, 5, 7, 9],
    "kneighborsclassifier__n_neighbors": [1],
    "kneighborsclassifier__metric":["cosine"]
}
models.append(("knn", knn_clf, parameters))


In [ ]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, pos_label=None, average='micro'),
           'recall': make_scorer(recall_score, pos_label=None, average='micro'),
           'f1_score': make_scorer(f1_score, pos_label=None, average='micro')}


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import fit_grid_point
from sklearn.pipeline import make_pipeline


In [ ]:
from sklearn import cross_validation
skf = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True, random_state=seed)
skf

In [ ]:
results = pd.DataFrame()
for name, model, parameter in models:
    print('modelo: ', name)
    print(parameter)
    
    tfidf=TfidfVectorizer(sublinear_tf=True, use_idf=True)
    pipe_model = make_pipeline(tfidf, model)
    gs = GridSearchCV(estimator=pipe_model, param_grid=parameter, scoring=scoring, refit='accuracy', cv = skf, return_train_score=True, verbose=5, n_jobs=4)
    gs.fit(X, y)
    print("Best Estimator: \n{}\n".format(gs.best_estimator_))
    print("Best Parameters: \n{}\n".format(gs.best_params_))
    print("Best Test Score: \n{}\n".format(gs.best_score_))
    d = gs.cv_results_
    d['metodo'] = name
    df = pd.DataFrame.from_dict(d)
    results = results.append(df, ignore_index=True)
    print(results)
    
    results.to_csv('resultados.csv', sep=';', decimal=',')
    print('_______________________________________________________')

In [ ]:
results